# Target Metadata: Coordinates, Priors, and Photometry

**Contents**
1. [Using Coordinates](#Using-Coordinates)
2. [Setting Initial Conditions](#Setting-Initial-Conditions)
3. [Adding/Removing Photometry](#Adding/Removing-Photometry)

The built-in `Target` class gives us a convenient infrastructure to change or view the metadata specific to our target in the simulation.

There are four main attribute categories that can be changed, and each may contain additional modifiable attributes.

The most basic way the `Target` class can be used is by initializing it with a name.

In [1]:
from stelpar import Target

target = Target('IRAS 04171+2756')

Now, `target` has been redifined as a `Target` object instead of just the target name string. It can be used in the estimation in exactly the same way as before, i.e., `est = Estimate(target)` (see, e.g., the [Quickstart Tutorial](./quickstart.ipynb)). Since for now we only set the `Target` object with a target name, it behaves the same within the simulation as setting `target = 'IRAS 04171+2756'`. 

## Using Coordinates

It's possible to pass coordinates for your target if you don't have a name that's recognizable by the databases. When you initialize a `Target` object, you can pass either an `astropy.coordinates.SkyCoord` object, or you can pass the coordinates as a tuple and any keyword arguments that you would use to create a `SkyCoord` object. See the [astropy documentation](https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) for more details.

We still need to give it a name, but now we can call it whatever we want since the name won't be used to query the photometry.

Let's create an arbitrary target called "my target" which we'll say has a right ascension and declination of 100 degrees and -50 degrees, respectively. Once it's initialized, we can treat it exactly as any other `Target` object.

In [2]:
my_target = Target('my target', (100, -50), unit='deg')
my_target.coords

<SkyCoord (ICRS): (ra, dec) in deg
    (100., -50.)>

## Setting Initial Conditions

The second metadata category includes all of the initial/boundary conditions for the simulation. There are four initial condition attributes that we can manipulate: bounds, prior, position, and perturbation. 

- *Bounds:* the hard boundaries of the simulation, beyond which the log-probability goes to $-\infty$ (i.e., the answer cannot lie outside the bounds). Each fit parameter (age, mass, $A_v$, f) has upper and lower bounds that can be easily changed if necessary. Default bounds for age and mass are constrained by the isochronal model that is used by the simulation. Default bounds for $A_v$ and f are loose physical constraints. Bounds are given as (upper, lower) pairs.

- *Prior:* the Gaussian (normal) distributions for any of the fit parameters as a way to constrain them. You can also set a prior for $T_{\mathrm{eff}}$, which is applied as an additional prior to age and mass since these two parameters are used to interpolate $T_{\mathrm{eff}}$ from the model grid. Priors are given as ($\mu$, $\sigma$) pairs, where $\mu$ and $\sigma$ are the mean and standard deviation of the distribution, respectively. Priors are all user-defined, thus none are applied by default.

- *Position and Perturbation:* these attributes combine to form the initial position for each fit parameter of the MCMC simulation. The initial position of each walker (in parameter space) is sampled from a normal distribution centered on the given position and scaled by the perturbation.

For reference, we can view the default initial conditions in `target`.

In [3]:
target.initial_conditions

bounds       prior position perturbation
target          parameter                                                  
IRAS 04171+2756 age        (1.0, 10000.0)  (nan, nan)      700           35
                mass         (0.09, 2.45)  (nan, nan)        1          0.2
                Av             (0.0, 3.0)  (nan, nan)        0          0.2
                f              (0.0, 2.0)  (nan, nan)        0          0.2
                Teff           (nan, nan)  (nan, nan)      NaN          NaN

López-Valdivia et al. 2021 (DOI: [10.3847/1538-4357/ac1a7b](https://dx.doi.org/10.3847/1538-4357/ac1a7b)) gives $T_{\mathrm{eff}} = 3407 \pm 198 \, \mathrm{K}$ for this star. We also know this is a star in the Taurus Star Forming Region (1-5 Myr), so let's say $\mathrm{age} = 3 \pm 2 \, \mathrm{Myr}$ (we won't change the bounds here so the walkers explore the parameter space). Let's put priors on these parameters to constrain the final result of the MCMC. We can also lower the initial position and perturbation of the age, but this isn't necessary if there are enough steps in the MCMC. Finally, since Taurus is an active star forming region, let's allow $A_v$ to be higher than 3 mag. These initial conditions are set using nested dictionaries for each attribute, and after we set them, we can check them again to make sure they've been applied.

In [4]:
target.initial_conditions = {
    'prior' : {
        'age':(3, 2),
        'Teff':(3407, 198)
    },
    'position' : {
        'age':20
    },
    'perturbation' : {
        'age':10
    },
    'bounds': {
        'Av':(0, 6)
    }
}

target.initial_conditions

bounds        prior position perturbation
target          parameter                                                   
IRAS 04171+2756 age        (1.0, 10000.0)       (3, 2)       20           10
                mass         (0.09, 2.45)   (nan, nan)        1          0.2
                Av                 (0, 6)   (nan, nan)        0          0.2
                f              (0.0, 2.0)   (nan, nan)        0          0.2
                Teff           (nan, nan)  (3407, 198)      NaN          NaN

## Adding/Removing Photometry

**Adding Photometry**

Depending on the target, it might be useful to add or remove photometry. You might prefer a different catalog that contains photometry that you'd like to use. Or you might have your own photometry that you've taken that you prefer to use over the existing values. If you find that the photometry is saturated or wrong in one or multiple bands you can remove these as well.

First let's look at the photometry that `stelpar` tries to collect for each target by default.

In [5]:
target.photometry_meta

magnitude    error system isochrone_analog
catalog        band                                                      
II/246/out     2mass_jmag           Jmag   e_Jmag   VEGA       2mass_jmag
               2mass_hmag           Hmag   e_Hmag   VEGA       2mass_hmag
               2mass_kmag           Kmag   e_Kmag   VEGA       2mass_kmag
I/355/gaiadr3  gaia_gmag            Gmag   e_Gmag   VEGA        gaia_gmag
               gaia_bpmag          BPmag  e_BPmag   VEGA       gaia_bpmag
               gaia_rpmag          RPmag  e_RPmag   VEGA       gaia_rpmag
V/154/sdss16   sdss_gmag            gmag   e_gmag     AB        sdss_gmag
               sdss_rmag            rmag   e_rmag     AB        sdss_rmag
               sdss_imag            imag   e_imag     AB        sdss_imag
               sdss_zmag            zmag   e_zmag     AB        sdss_zmag
II/336/apass9  johnson_bmag         Bmag   e_Bmag   VEGA     johnson_bmag
               johnson_vmag         Vmag   e_Vmag   VEGA     johnson_vmag
I/259/tyc2     tycho_bmag          BTmag  e_BTmag   VEGA       tycho_bmag
               tycho_vmag          VTmag  e_VTmag   VEGA       tycho_vmag
I/239/hip_main hipparcos_hpmag     Hpmag  e_Hpmag   VEGA  hipparcos_hpmag

Suppose we want to add BT and VT magnitudes from Hipparcos if they're available (in addition to the ones from TYCHO). We would pass a nested `dict` to the `add_photometry` method of our `Target` instance. The first level is the name of the catalog that is queried. The second level contains the names that we assign to the bands. These names are only relevant in our photometry `DataFrame` (of the `Estimate` class) so they can be whatever we want them to be. The important information is contained beyond these band keys in a `tuple` which contains four strings. The first two are the column names in the queried catalog of the magnitude and error. The third is the magnitude system (either 'vega' or 'AB'). The fourth value tells `stelpar` which of the bands in the isochronal grid is analogous to this new band. This is an optional requirement, but highly recommended. If no analog is given, the band name is used. The list of available analogs is given by the `isochrone_analogs` attribute of our `Target` instance.

In [6]:
target.add_photometry(
    {
        'I/239/hip_main' : {
            'hipparcos_btmag' : ('BTmag', 'e_BTmag', 'vega', 'tycho_bmag'),
            'hipparcos_vtmag' : ('VTmag', 'e_VTmag', 'vega', 'tycho_vmag'),
        }
    }
)

print("Isochrone analogs:\n")
print(target.isochrone_analogs)

Isochrone analogs:

['2mass_hmag', '2mass_jmag', '2mass_kmag', 'cousins_imag', 'cousins_rmag', 'gaia_bpmag', 'gaia_gmag', 'gaia_rpmag', 'hipparcos_hpmag', 'johnson_bmag', 'johnson_vmag', 'ps1_gmag', 'ps1_imag', 'ps1_omag', 'ps1_rmag', 'ps1_wmag', 'ps1_ymag', 'ps1_zmag', 'sdss_gmag', 'sdss_imag', 'sdss_rmag', 'sdss_zmag', 'tycho_bmag', 'tycho_vmag']


**Local Photometry**

Sometimes we might have our own photometry that we've taken that we'd like to use. These can be added in a similar way as before, except the catalog must be `'local'`, and the `magnitude` and `error` values in the tuple must be floats.

Suppose we've measured R and I magnitudes to be 14 and 15 mag, respectively, and each with an error of 0.05 mag. Remember we can name these whatever we want as long as they have an appropriate analog in the isochrone model.

In [7]:
target.add_photometry(
    {
        'local' : {
             'my_rmag' : (14, 0.05, 'vega', 'cousins_rmag'),
             'my_imag' : (15, 0.05, 'vega', 'cousins_imag')
        }
    }
)

**Removing Photometry**

Just as we can add photometry, we can also remove any catalogs/bands from being included in the photometry search. This may be useful if, for example, the photometry is saturated in a particular band or for some reason it's obvious the photometry is wrong. Removing photometry is simpler than adding since all we're doing is deleting rows from the above table.

We'll use the `remove_photometry` method of our `Target` object and pass a dictionary whose keys are the catalogs and whose values are either the band, list of bands, or 'all'. The first two are used for removing an individual band(s) and the third is for removing the whole catalog.

Let's suppose we want to remove 2MASS H, SDSS g, SDSS z, and both Johnson magnitudes from being collected.

In [8]:
target.remove_photometry(
    {
        'II/246/out' : '2mass_hmag',
        'V/154/sdss16' : ['sdss_gmag', 'sdss_rmag'],
        'II/336/apass9' : 'all'
    }
)

Finally let's check that all of these changes have been applied.

In [9]:
target.photometry_meta

magnitude    error system isochrone_analog
catalog        band                                                      
II/246/out     2mass_jmag           Jmag   e_Jmag   VEGA       2mass_jmag
               2mass_kmag           Kmag   e_Kmag   VEGA       2mass_kmag
I/355/gaiadr3  gaia_gmag            Gmag   e_Gmag   VEGA        gaia_gmag
               gaia_bpmag          BPmag  e_BPmag   VEGA       gaia_bpmag
               gaia_rpmag          RPmag  e_RPmag   VEGA       gaia_rpmag
V/154/sdss16   sdss_imag            imag   e_imag     AB        sdss_imag
               sdss_zmag            zmag   e_zmag     AB        sdss_zmag
I/259/tyc2     tycho_bmag          BTmag  e_BTmag   VEGA       tycho_bmag
               tycho_vmag          VTmag  e_VTmag   VEGA       tycho_vmag
I/239/hip_main hipparcos_hpmag     Hpmag  e_Hpmag   VEGA  hipparcos_hpmag
               hipparcos_btmag     BTmag  e_BTmag   vega       tycho_bmag
               hipparcos_vtmag     VTmag  e_VTmag   vega       tycho_vmag
local          my_rmag                14     0.05   vega     cousins_rmag
               my_imag                15     0.05   vega     cousins_imag

We can see that BT and VT magnitudes were added from Hipparcos, local R and I photometry were added, and 2MASS H, SDSS g, SDSS z, and both Johnson magnitudes were removed.

<div class="alert alert-block alert-info">
<b>Reminder:</b> Adding photometry does not guarantee it will be used in the simulation, only that it will be included in the photometry query. If photometry cannot be found in the default or added catalogs, it will not be used (unless it's given as 'local' photometry).
</div>